In [ ]:
from typing import List


class Solution:

    def smallestSufficientTeam(self, req_skills: List[str], people: List[List[str]]) -> List[int]:
        ha = {x: i for i, x in enumerate(req_skills)}
        m, n = len(req_skills), len(people)
        mask = [0] * n
        for i in range(n):
            for p in people[i]:
                mask[i] |= (1 << ha[p])

        f = [(1 << n) - 1] * (1 << m)
        f[0] = 0
        for i in range(n):
            for j in range((1 << m) - 1, -1, -1):
                # f[j] = min(f[j], f[j & ~mask[i]] | (1 << i), key = lambda x:x.bit_count())
                sub = f[j & ~mask[i]] | (1 << i)
                if sub.bit_count() < f[j].bit_count():
                    f[j] = sub
        return [i for i in range(n) if f[(1 << m) - 1] & (1 << i)]

        # f = [[(1 << n) - 1] * (1 << m) for _ in range(n + 1)]
        # f[0][0] = 0
        # for i in range(n):
        #     for j in range(1 << m):
        #         sub1 = f[i][j]
        #         sub2 = f[i][j & ~mask[i]] | (1 << i)
        #         f[i + 1][j] = sub1 if sub1.bit_count() < sub2.bit_count() else sub2
        # return [i for i in range(n) if f[n][(1 << m) - 1] & (1 << i)]

        # 01背包
        # @lru_cache(None)
        # def dfs(i, j): # 从前i个mask 取出并集为j 的最小集合
        #     if i < 0:
        #         return 0 if j == 0 else (1 << n) - 1
        #     # 不取
        #     sub1 = dfs(i - 1, j)
        #     # 取
        #     sub2 = dfs(i - 1, j & ~mask[i]) | (1 << i)
        #     return sub1 if sub1.bit_count() < sub2.bit_count() else sub2
        # res = dfs(n - 1, (1 << m) - 1)
        # return [i for i in range(n) if res & (1 << i)]

In [ ]:
class Solution:

    def smallestSufficientTeam(self, req_skills: List[str], people: List[List[str]]) -> List[int]:
        ha = {x: i for i, x in enumerate(req_skills)}
        m, n = len(req_skills), len(people)
        mask = [0] * n
        for i in range(n):
            for p in people[i]:
                mask[i] |= (1 << ha[p])

        @lru_cache(None)
        def dfs(i, j):  # 从前i个mask 取出并集为j 的最小长度
            if i < 0:
                return 0 if j == 0 else n # inf
            # 不取
            sub1 = dfs(i - 1, j)
            # 取
            sub2 = dfs(i - 1, j & ~mask[i]) + 1
            return min(sub1, sub2)

        def make(i, j):  # 从前i个mask 取出并集为j 的最小二进制集合
            if i < 0:
                return 0 if j == 0 else (1 << n) - 1
            if dfs(i - 1, j) < dfs(i - 1, j & ~mask[i]) + 1:  # 不取可以得到更短
                return make(i - 1, j)
            return make(i - 1, j & ~mask[i]) | (1 << i)

        return [i for i in range(n) if make(n - 1, (1 << m) - 1) & (1 << i)]